# PDF to Podcast

## Prerequisites

1. Create a virtual environment and install the required packages.

    ```bash
    python -m venv venv
    ```

2. Install the required packages:

In [24]:
pip install -qU autogen pypdf langchain langchain-text-splitters langchain-core langchain-community lancedb langchain-openai python-dotenv azure-cognitiveservices-speech

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Read .env file

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

def get_file(file_name: str):
  """Get file path

  Args:
      file_name (str): File name

  Returns:
      File path
  """
  output_folder = 'outputs'
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  return os.path.join(output_folder, file_name)

## PDF Information

In [3]:
# Set the pdf information

pdf_title = 'LoRA: Low-Rank Adaptation of Large Language Models'
pdf_url = 'https://arxiv.org/pdf/2106.09685'

In [4]:
# Define the file name without special characters

pdf_filename = pdf_title.replace(':', '').replace('-', '') + '.pdf'
print(pdf_filename)


LoRA LowRank Adaptation of Large Language Models.pdf


## Load PDF as langchain document

In [5]:
# Download PDF file

import requests

response = requests.get(pdf_url)

with open(get_file(pdf_filename), 'wb') as file:
  file.write(response.content)



In [6]:
# Create the documents from the PDF file

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(get_file(pdf_filename))
documents = loader.load()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7c313933d2e0>>
Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [8]:
# Update the metadata of all documents

for document in documents:
  document.metadata['title'] = pdf_title
  document.metadata['source'] = pdf_url
  document.metadata['description'] = ''
  document.metadata['thumbnail_url'] = ''
  document.metadata['type'] = 'pdf'

## Create embeddings

In [9]:
# Split the document in chunks of maximum 1000 characters with 200 characters overlap using langchain

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200
)
splits = text_splitter.split_documents(documents)

In [10]:
# Define the embeddings model

from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
  api_key=os.environ['OPENAI_API_KEY'],
  azure_endpoint=os.environ['OPENAI_AZURE_ENDPOINT'],
  api_version=os.environ['OPENAI_API_VERSION'],
  azure_deployment=os.environ['OPENAI_AZURE_DEPLOYMENT_EMBEDDINGS']
)

In [11]:
azure_openai_embeddings

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7bda92f7c140>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7bda92f7d6a0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-3-large', openai_api_version='2024-02-15-preview', openai_api_base=None, openai_api_type='azure', openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=2048, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True, azure_endpoint='https://ipej-hackathon24-aoi.openai.azure.com/', azure_ad_token=None, azure_ad_token_provider=None, validate_base_url=True)

In [28]:
# Create the vector store

import lancedb
from langchain_community.vectorstores import LanceDB

db = lancedb.connect("/tmp/lancedb")

vectorstore = LanceDB.from_documents(
  documents=splits,
  embedding=azure_openai_embeddings
)

retriever = vectorstore.as_retriever()

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [13]:
# Clean up: delete the downloaded PDF file

os.remove(get_file(pdf_filename))

## Create the langchain chain to do RAG

In [29]:
# Create the prompt for the chain with embeddings and LLM

from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
# Define the LLM model

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
  azure_endpoint=os.environ['OPENAI_AZURE_ENDPOINT'],
  api_version=os.environ['OPENAI_API_VERSION'],
  azure_deployment=os.environ['OPENAI_AZURE_DEPLOYMENT'],
  temperature=0,
  top_p=1
)

In [31]:
# Define the rag chain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

NameError: name 'retriever' is not defined

## Create the outline of the podcast

In [32]:
podcast_outline_response = rag_chain.invoke({"input": "Create an outline for a podcast on LoRA."})
podcast_outline = podcast_outline_response['answer']

NameError: name 'rag_chain' is not defined

In [33]:
# Write the podcast outline

podcast_outline_file_name = pdf_filename.replace('.pdf', '_script.txt')

with open(get_file(podcast_outline_file_name), "w") as f:
    f.write(podcast_outline)

NameError: name 'podcast_outline' is not defined

## Create the podcast script

In [34]:
def read_and_print_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        return content
        #print(content)

# Example usage
file_path = '/workspaces/AutoPodCaster/data/How to fine-tune a model using LoRA (step by step)_podcast_outline.txt'  # Replace with the path to your file
podcast_outline = read_and_print_file(file_path)


FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/AutoPodCaster/data/How to fine-tune a model using LoRA (step by step)_podcast_outline.txt'

In [22]:
segments_outline = podcast_outline.split("####")
print(segments_outline)

['### Podcast Outline: How to Fine-Tune a Model Using LoRA (Step by Step)\n\n', " Introduction\n1. **Welcome and Introduction**\n   - Brief introduction to the podcast and today's topic.\n   - Overview of what listeners will learn about fine-tuning models using LoRA.\n\n2. **What is LoRA?**\n   - Explanation of LoRA (Low-Rank Adaptation).\n   - Benefits of using LoRA for fine-tuning models.\n   - Importance of fine-tuning large models for specific tasks.\n\n", ' Segment 1: Setting Up for Fine-Tuning\n1. **Choosing the Right Hardware**\n   - Importance of using a GPU for fine-tuning.\n   - Example of using an L4 GPU and its cost-effectiveness.\n\n2. **Loading the Base Model**\n   - Steps to load the base model into memory.\n   - Printing the number of parameters in the base model.\n\n', ' Segment 2: Understanding LoRA Configuration\n1. **Decomposing Matrices**\n   - Explanation of rank one matrices and their role in LoRA.\n   - Trade-offs between lower and higher ranks in terms of accur

In [38]:
from IPython.display import Image, display

import autogen
from autogen.coding import LocalCommandLineCodeExecutor

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST"
)

In [ ]:
for segment in range(len(segments_outline)):
    # Create a prompt with the outline to get a full podcast text

    podcast_prompt = f"""Create a podcast complete text based on the following outline:

    {segment}

    This text will be used to generate the audio of the podcast. There are 2 participants in the podcast: the host and the guest. The host will introduce the podcast and the guest. The guest will explain the outline of the podcast. The host will ask questions to the guest and the guest will answer them. The host will thank the guest and close the podcast.
    The name of the host is Pierre and his role is to be the listener's podcast assistant. The name of the guest is Marie and her role is to be the expert in the podcast topic. The name of the podcast is "Advanced AI Podcast".

    When you thanks someone, write "Thank you" and the name of the person without a comma. For example, "Thank you Pierre".

    Output as a JSON with the following fields:
    - title: Title of the podcast
    - text: an array of objects with the speaker, the intonation and the text to be spoken
    Return only the json as plain text.
    """

    formatted_podcast_prompt = podcast_prompt.format(segment)

    podcast_script_response = rag_chain.invoke({"input": formatted_podcast_prompt})
    podcast_script_text = podcast_script_response['answer']
    print(podcast_script_text)
    

In [59]:
import json
import os

# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)

def my_message_generator(sender, recipient, context):
    json_file_name = r"C:\Users\kgellynck\AutoPodcast\AutoPodCaster\data\How to fine-tune a model using LoRA (step by step)_podcast_script.json"
    txt_file_name = r"C:\Users\kgellynck\AutoPodcast\AutoPodCaster\data\podcast_script.txt"
    
    # Check if the JSON file exists
    if not os.path.exists(json_file_name):
        return "JSON file not found."
    
    try:
        with open(json_file_name, mode="r", encoding="utf-8") as json_file:
            podcast_script_json = json.load(json_file)
    except FileNotFoundError:
        return "No data found."
    except json.JSONDecodeError as e:
        return f"Error parsing JSON: {e}"
    
    # Extract the full text variable
    full_text = podcast_script_json.get("text", [])
    
    # Extract conversation between Pierre and Marie and write to a text file
    try:
        with open(txt_file_name, mode="w", encoding="utf-8") as txt_file:
            for entry in full_text:
                if isinstance(entry, dict):
                    speaker = entry.get("speaker", "")
                    text = entry.get("text", "")
                    if speaker in ["Pierre", "Marie"]:
                        # Handle None values
                        speaker = speaker if speaker is not None else "Unknown"
                        text = text if text is not None else ""
                        txt_file.write(f"{speaker}: {text}\n")
                else:
                    return "Unexpected JSON structure."
    except Exception as e:
        return f"Error writing to text file: {e}"
    
    return "Read the text and see if the conversation is a natural one"+txt_file_name


# followup of the previous question
chat_res = user_proxy.initiate_chat(
    recipient=assistant,
    message=my_message_generator,
    summary_method="reflection_with_llm"
)

JSON content has been converted to text and saved to C:\Users\kgellynck\AutoPodcast\AutoPodCaster\data\podcast_script.txt


In [ ]:
import json
import os

# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)

def my_message_generator(sender, recipient, context):
    json_file_name = r"C:\Users\kgellynck\AutoPodcast\AutoPodCaster\data\How to fine-tune a model using LoRA (step by step)_podcast_script.json"
    txt_file_name = r"C:\Users\kgellynck\AutoPodcast\AutoPodCaster\data\podcast_script.txt"
    
    # Check if the JSON file exists
    if not os.path.exists(json_file_name):
        return "JSON file not found."
    
    try:
        with open(json_file_name, mode="r", encoding="utf-8") as json_file:
            podcast_script_json = json.load(json_file)
    except FileNotFoundError:
        return "No data found."
    except json.JSONDecodeError as e:
        return f"Error parsing JSON: {e}"
    
    # Extract conversation between Pierre and Marie and write to a text file
    try:
        with open(txt_file_name, mode="w", encoding="utf-8") as txt_file:
            for entry in podcast_script_json:
                if isinstance(entry, dict):
                    speaker = entry.get("speaker", "Unknown")
                    text = entry.get("text", "")
                    if speaker in ["Pierre", "Marie"]:
                        txt_file.write(f"{speaker}: {text}\n")
                else:
                    return "Unexpected JSON structure."
    except Exception as e:
        return f"Error writing to text file: {e}"
    
    return f"JSON content has been converted to text and saved to {txt_file_name}"

# followup of the previous question
chat_res = user_proxy.initiate_chat(
    recipient=assistant,
    message=my_message_generator,
    summary_method="reflection_with_llm"
)

In [ ]:
file_name = r"C:\Users\kgellynck\AutoPodcast\AutoPodCaster\data\How to fine-tune a model using LoRA (step by step)_podcast_script.json"

In [79]:
# Create a prompt with the outline to get a full podcast text

podcast_prompt = f"""Create a podcast complete text based on the following outline:

{podcast_outline}

This text will be used to generate the audio of the podcast. There are 2 participants in the podcast: the host and the guest. The host will introduce the podcast and the guest. The guest will explain the outline of the podcast. The host will ask questions to the guest and the guest will answer them. The host will thank the guest and close the podcast.
The name of the host is Pierre and his role is to be the listener's podcast assistant. The name of the guest is Marie and her role is to be the expert in the podcast topic. The name of the podcast is "Advanced AI Podcast".

When you thanks someone, write "Thank you" and the name of the person without a comma. For example, "Thank you Pierre".

Output as a JSON with the following fields:
- title: Title of the podcast
- text: an array of objects with the speaker, the intonation and the text to be spoken
Return only the json as plain text.
"""

formatted_podcast_prompt = podcast_prompt.format(podcast_outline)

In [80]:
# Generate the podcast script

podcast_script_response = rag_chain.invoke({"input": formatted_podcast_prompt})
podcast_script_text = podcast_script_response['answer']

In [81]:
# Save the podcast script

podcast_script_file_name = pdf_filename.replace('.pdf', '_script.json')

with open(get_file(podcast_script_file_name), "w") as f:
    f.write(podcast_script_text)

## Generate the podcast audio

In [82]:
import azure.cognitiveservices.speech as speechsdk
import json

# Creates an instance of a speech config with specified subscription key and service region.
speech_key = os.environ['AZURE_SPEECH_KEY']
service_region = os.environ['AZURE_SPEECH_REGION']

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# This is an example of SSML (Speech Synthesis Markup Language) format.
# <speak version="1.0" xmlns="https://www.w3.org/2001/10/synthesis" xml:lang="en-US">
#   <voice name="en-US-AvaMultilingualNeural">
#     When you're on the freeway, it's a good idea to use a GPS.
#   </voice>
# </speak>
# Parse the JSON response and create a SSML with en-US-GuyNeural for Pierre Voice
# and en-US-JennyNeural for Marie Voice
podcast_script_json = json.loads(str(podcast_script_text))
ssml_text = "<speak version='1.0' xmlns='https://www.w3.org/2001/10/synthesis' xml:lang='en-US'>"
for line in podcast_script_json['text']:
    speaker = line['speaker']
    text = line['text']
    if speaker == 'Pierre':
        ssml_text += f"<voice name='en-US-GuyNeural'>{text}</voice>"
    elif speaker == 'Marie':
        ssml_text += f"<voice name='en-US-JennyNeural'>{text}</voice>"
ssml_text += "</speak>"

# use the default speaker as audio output.
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

result = speech_synthesizer.speak_ssml_async(ssml_text).get()
stream = speechsdk.AudioDataStream(result)
podcast_filename = pdf_filename.replace('.pdf', '_podcast.wav')
stream.save_to_wav_file(get_file(podcast_filename))



ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
